## Modeling

For background on this project, please see the [README](../README.md).

**Notebooks**
- [Data Acquisition & Cleaning](./01_data_acq_clean.ipynb)
- [Exploratory Data Analysis](./02_eda.ipynb)
- Modeling (this notebook)
- [Results and Recommendations](./04_results.ipynb)

**In this notebook, you'll find:**
- Classification models using content-based filtering
- Brief analyses for each model

In [187]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

from scipy import sparse, spatial
import sys
from sklearn.metrics.pairwise import pairwise_distances, cosine_similarity, cosine_distances

from numpy import dot
from numpy.linalg import norm

---
#### **Importing All-Feature Dataframe**

This is the aggregated dataframe of all of our datasets.

In [188]:
model_df = pd.read_csv('../datasets/cleaned_data/all_features.csv')

model_df.head()

,place,state,price,auto_accss_idx,pct_no_vehicle,pct_one_or_more_vehicles,transit_accss_idx,walkability_idx,cost_of_living,grocery_cost_idx,...,nov_avg_rain,dec_avg_rain,winter_avg_temp,spring_avg_temp,summer_avg_temp,autumn_avg_temp,winter_avg_rain,spring_avg_rain,summer_avg_rain,autumn_avg_rain
0,Asheville,NC,225.830385,0.580614,0.053756,0.946244,0.149850,9.382035,105.7,103.1,...,3.11,3.69,40.166667,53.333333,70.200000,55.333333,3.710000,4.160000,4.713333,3.423333
1,Austin,TX,436.273137,0.718323,0.059187,0.940813,0.286952,11.516379,129.4,96.7,...,2.52,2.42,53.400000,67.433333,82.333333,70.366667,2.243333,3.256667,2.386667,3.026667
2,Boston,MA,272.879054,0.759096,0.301252,0.698748,0.409262,14.936791,153.4,111.4,...,4.04,4.01,33.100000,49.700000,72.233333,56.000000,3.810000,3.790000,3.463333,3.703333
3,Cambridge,MA,246.035242,0.563985,0.102769,0.897231,0.160956,12.458517,173.7,115.5,...,4.00,3.85,31.100000,49.166667,71.633333,54.533333,3.570000,3.706667,3.523333,3.676667
4,Chicago,IL,245.904608,0.636256,0.169022,0.830978,0.310866,13.635529,107.4,97.2,...,2.44,2.04,28.133333,50.833333,73.933333,55.766667,1.760000,3.223333,3.643333,2.853333


In [189]:
model_df.columns

Index(['place', 'state', 'price', 'auto_accss_idx', 'pct_no_vehicle',
       'pct_one_or_more_vehicles', 'transit_accss_idx', 'walkability_idx',
       'cost_of_living', 'grocery_cost_idx', 'transportation_cost_idx',
       'miscellaneous_cost_idx', 'population', 'median_age',
       'summer_comfort_index', 'winter_comfort_index',
       'perc_growth_since_2020', 'political_left_lean', 'political_right_lean',
       'chain_ratio', 'jan_avg_temp', 'feb_avg_temp', 'march_avg_temp',
       'april_avg_temp', 'may_avg_temp', 'june_avg_temp', 'july_avg_temp',
       'aug_avg_temp', 'sept_avg_temp', 'oct_avg_temp', 'nov_avg_temp',
       'dec_avg_temp', 'jan_avg_rain', 'feb_avg_rain', 'mar_avg_rain',
       'apr_avg_rain', 'may_avg_rain', 'jun_avg_rain', 'jul_avg_rain',
       'aug_avg_rain', 'sep_avg_rain', 'oct_avg_rain', 'nov_avg_rain',
       'dec_avg_rain', 'winter_avg_temp', 'spring_avg_temp', 'summer_avg_temp',
       'autumn_avg_temp', 'winter_avg_rain', 'spring_avg_rain',
       'sum

In [190]:
model_df['place'] = model_df[['place', 'state']].apply(lambda x: ', '.join(x), axis=1)
model_df.drop(columns='state', inplace=True)
model_df.head()
# model_df = model_df[['']]

,place,price,auto_accss_idx,pct_no_vehicle,pct_one_or_more_vehicles,transit_accss_idx,walkability_idx,cost_of_living,grocery_cost_idx,transportation_cost_idx,...,nov_avg_rain,dec_avg_rain,winter_avg_temp,spring_avg_temp,summer_avg_temp,autumn_avg_temp,winter_avg_rain,spring_avg_rain,summer_avg_rain,autumn_avg_rain
0,"Asheville, NC",225.830385,0.580614,0.053756,0.946244,0.149850,9.382035,105.7,103.1,82.8,...,3.11,3.69,40.166667,53.333333,70.200000,55.333333,3.710000,4.160000,4.713333,3.423333
1,"Austin, TX",436.273137,0.718323,0.059187,0.940813,0.286952,11.516379,129.4,96.7,109.9,...,2.52,2.42,53.400000,67.433333,82.333333,70.366667,2.243333,3.256667,2.386667,3.026667
2,"Boston, MA",272.879054,0.759096,0.301252,0.698748,0.409262,14.936791,153.4,111.4,131.8,...,4.04,4.01,33.100000,49.700000,72.233333,56.000000,3.810000,3.790000,3.463333,3.703333
3,"Cambridge, MA",246.035242,0.563985,0.102769,0.897231,0.160956,12.458517,173.7,115.5,95.7,...,4.00,3.85,31.100000,49.166667,71.633333,54.533333,3.570000,3.706667,3.523333,3.676667
4,"Chicago, IL",245.904608,0.636256,0.169022,0.830978,0.310866,13.635529,107.4,97.2,138.5,...,2.44,2.04,28.133333,50.833333,73.933333,55.766667,1.760000,3.223333,3.643333,2.853333


In [191]:
trial_model_df = model_df[['place', 'winter_avg_temp', 'summer_avg_temp', 'chain_ratio', 'walkability_idx', 'political_left_lean', 'political_right_lean', 'miscellaneous_cost_idx', 'price']]

In [56]:
trial_model_df.drop(33, inplace=True)

/var/folders/td/_057xlcn5px5szx_85_2v2dc0000gn/T/ipykernel_1861/91443834.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trial_model_df.drop(33, inplace=True)


In [192]:
# Converting price column to monthly cost with Airbnb's estimated 30+day-stay discount
trial_model_df['avg_monthly_price'] = (trial_model_df['price'] * 30) - ((trial_model_df['price'] * 30) * 0.25)
trial_model_df.drop(columns=['price'], inplace=True)
trial_model_df.head()

/var/folders/td/_057xlcn5px5szx_85_2v2dc0000gn/T/ipykernel_1861/397421596.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trial_model_df['avg_monthly_price'] = (trial_model_df['price'] * 30) - ((trial_model_df['price'] * 30) * 0.25)
/var/folders/td/_057xlcn5px5szx_85_2v2dc0000gn/T/ipykernel_1861/397421596.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trial_model_df.drop(columns=['price'], inplace=True)


,place,winter_avg_temp,summer_avg_temp,chain_ratio,walkability_idx,political_left_lean,political_right_lean,miscellaneous_cost_idx,avg_monthly_price
0,"Asheville, NC",40.166667,70.200000,0.3516,9.382035,0.597,0.386,111.2,5081.183654
1,"Austin, TX",53.400000,82.333333,0.3814,11.516379,0.714,0.264,105.9,9816.145585
2,"Boston, MA",33.100000,72.233333,0.2796,14.936791,0.806,0.175,157.9,6139.778716
3,"Cambridge, MA",31.100000,71.633333,0.3277,12.458517,0.715,0.263,176.7,5535.792952
4,"Chicago, IL",28.133333,73.933333,0.3717,13.635529,0.742,0.240,139.8,5532.853672


In [193]:
trial_model_df

,place,winter_avg_temp,summer_avg_temp,chain_ratio,walkability_idx,political_left_lean,political_right_lean,miscellaneous_cost_idx,avg_monthly_price
0,"Asheville, NC",40.166667,70.200000,0.3516,9.382035,0.5970,0.3860,111.20,5081.183654
1,"Austin, TX",53.400000,82.333333,0.3814,11.516379,0.7140,0.2640,105.90,9816.145585
2,"Boston, MA",33.100000,72.233333,0.2796,14.936791,0.8060,0.1750,157.90,6139.778716
3,"Cambridge, MA",31.100000,71.633333,0.3277,12.458517,0.7150,0.2630,176.70,5535.792952
4,"Chicago, IL",28.133333,73.933333,0.3717,13.635529,0.7420,0.2400,139.80,5532.853672
5,"Columbus, OH",32.866667,74.100000,0.5012,11.556746,0.6470,0.3340,107.50,3835.510118
6,"Dallas, TX",49.566667,82.666667,0.4783,11.899341,0.6490,0.3330,107.80,5658.053279
7,"Denver, CO",33.166667,73.466667,0.3274,14.462231,0.7960,0.1820,152.20,4313.462492
8,"Fort Lauderdale, FL",70.300000,82.533333,0.3837,12.445745,0.6450,0.3470,107.60,5663.896761
9,"Fort Worth, TX",49.166667,82.133333,0.5595,9.585953,0.4930,0.4910,109.20,4079.534543


---
#### **Building a Function to Recommend a Location Based on User Input**

In [243]:
user_input = [65, 75, 'I prefer non-chain restaurants', 'Neutral', 'Moderate left lean preferred', 5, 6000]
user_input

[65,
 75,
 'I prefer non-chain restaurants',
 'Neutral',
 'Moderate left lean preferred',
 5,
 6000]

In [112]:
user_input.insert(0, 'User Input')

In [244]:
user_input

[65,
 75,
 'I prefer non-chain restaurants',
 'Neutral',
 'Moderate left lean preferred',
 5,
 6000]

In [239]:
# trial_model_df.drop(33, inplace=True)

In [247]:
trial_model_df.to_csv('../streamlit/test_df.csv', index=False)

In [245]:
def wfh(user_input_data):
    '''Inserting placeholder for location name'''
    user_input_data.insert(0, 'User Input')

    '''Converting user list to dictionary'''
    feat_keys = ['place', 'low_temp', 'high_temp', 'food_pref', 'walk_pref', 'pol_dem_pref', 'misc_costs', 'monthly_budget']
    user_input_dict = dict(zip(feat_keys, user_input_data))

    '''Creating indices for ordinal input values'''

    # Food preferences
    chain_mean = trial_model_df['chain_ratio'].mean()
    chain_std = trial_model_df['chain_ratio'].std()

    chain_input_options = ['I only eat at Mom-n-Pop restaurants', 'I prefer non-chain restaurants', 'I like a combination of both', 'I would rather go somewhere that has a drive-thru', 'The faster the food the better']
    chain_input_index = [chain_mean - (2*chain_std), chain_mean - (1*chain_std), chain_mean, chain_mean + (1*chain_std), chain_mean + (2*chain_std)]
    chain_mapper = dict(zip(chain_input_options, chain_input_index))
    
    # Walkability
    walk_mean = trial_model_df['walkability_idx'].mean()
    walk_std = trial_model_df['walkability_idx'].std()

    walk_input_options = ['Not important at all', 'Not very important', 'Neutral', 'A little important', 'Very important']
    walk_input_index = [walk_mean - (2*walk_std), walk_mean - (1*walk_std), walk_mean, walk_mean + (1*walk_std), walk_mean + (2*walk_std)]
    walkability_mapper = dict(zip(walk_input_options, walk_input_index))

    # Political lean preferences
    # Using the mean here instead of 50/50 because most locations have a slight to moderate left lean, meaning that if we set the default value to 0.5, we'd be unintentionally biasing the results
    pol_mean = trial_model_df['political_left_lean'].mean()
    pol_std = trial_model_df['political_left_lean'].std()

    pol_pref_options = ['Strong left lean preferred', 'Moderate left lean preferred', 'Moderate right lean preferred', 'Strong right lean preferred']
    pol_pref_index = [pol_mean + (2*pol_std), pol_mean + (1*pol_std), pol_mean - (1*pol_std), pol_mean - (2*pol_std)]
    pol_pref_mapper = dict(zip(pol_pref_options, pol_pref_index))

    '''Mapping food preference to numerical value'''
    for k, v in chain_mapper.items():
        if user_input_dict['food_pref'] == k:
            user_input_dict['food_pref'] = v

    '''Mapping walkability preference to numerical value'''
    for k, v in walkability_mapper.items():
        if user_input_dict['walk_pref'] == k:
            user_input_dict['walk_pref'] = v

    '''Mapping political preference to numerical value'''
    for k, v in pol_pref_mapper.items():
        if user_input_dict['pol_dem_pref'] == k:
            user_input_dict['pol_dem_pref'] = v
        else:
            user_input_dict['pol_dem_pref'] = pol_mean
    
    pol_rep_pref = 1 - user_input_dict['pol_dem_pref']

    '''Estimating miscellaneous cost index based on user input'''
    misc_mean = trial_model_df['miscellaneous_cost_idx'].mean()
    misc_std = trial_model_df['miscellaneous_cost_idx'].std()

    user_input_dict['misc_costs'] = misc_mean + (user_input_dict['misc_costs'] * misc_std/5)

    '''Converting dictionary values to list'''
    user_input_list = list(user_input_dict.values())

    '''Adding a Repulican-lean political value'''
    user_input_list.insert(6, pol_rep_pref)

    '''Adding the updated list as a new row in our dataframe'''
    trial_model_df.loc[len(trial_model_df)] = user_input_list
    
    return user_input_list

In [254]:
trial_model_df[trial_model_df['place'] == 'Austin, TX']['avg_monthly_price']

1    9816.145585
Name: avg_monthly_price, dtype: float64

In [266]:
trial_model_df.loc[trial_model_df['place'] == 'Austin, TX'].iloc[:, -1:]

,avg_monthly_price
1,9816.145585


In [270]:
model_df.columns

Index(['place', 'price', 'auto_accss_idx', 'pct_no_vehicle',
       'pct_one_or_more_vehicles', 'transit_accss_idx', 'walkability_idx',
       'cost_of_living', 'grocery_cost_idx', 'transportation_cost_idx',
       'miscellaneous_cost_idx', 'population', 'median_age',
       'summer_comfort_index', 'winter_comfort_index',
       'perc_growth_since_2020', 'political_left_lean', 'political_right_lean',
       'chain_ratio', 'jan_avg_temp', 'feb_avg_temp', 'march_avg_temp',
       'april_avg_temp', 'may_avg_temp', 'june_avg_temp', 'july_avg_temp',
       'aug_avg_temp', 'sept_avg_temp', 'oct_avg_temp', 'nov_avg_temp',
       'dec_avg_temp', 'jan_avg_rain', 'feb_avg_rain', 'mar_avg_rain',
       'apr_avg_rain', 'may_avg_rain', 'jun_avg_rain', 'jul_avg_rain',
       'aug_avg_rain', 'sep_avg_rain', 'oct_avg_rain', 'nov_avg_rain',
       'dec_avg_rain', 'winter_avg_temp', 'spring_avg_temp', 'summer_avg_temp',
       'autumn_avg_temp', 'winter_avg_rain', 'spring_avg_rain',
       'summer_avg_r

In [271]:
date_str = '2023-01-01&'

print(date_str[5:7])

01


In [246]:
print(wfh(user_input))

['User Input', 65, 75, 0.22781856884367266, 12.07544654388207, 0.7012, 0.29879999999999995, 154.74250341119108, 6000]


/var/folders/td/_057xlcn5px5szx_85_2v2dc0000gn/T/ipykernel_1861/73214471.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trial_model_df.loc[len(trial_model_df)] = user_input_list


In [204]:
trial_model_df.columns[1:]

Index(['winter_avg_temp', 'summer_avg_temp', 'chain_ratio', 'walkability_idx',
       'political_left_lean', 'political_right_lean', 'miscellaneous_cost_idx',
       'avg_monthly_price'],
      dtype='object')

In [206]:
# Scaling all numeric columns
feature_cols = trial_model_df.columns[1:]
sc = MinMaxScaler()
scaled_df = sc.fit_transform(trial_model_df[feature_cols])

# Setting model and indices for reference
indices = pd.Series(trial_model_df.index, index=trial_model_df['place'])
cs_df = cosine_similarity(scaled_df)

In [231]:
def find_rec(input_row = 'User Input', model = cs_df):

    '''Indexing the user input row'''
    index = indices[input_row]

    '''Filtering and sorting the cosine similarity values for input data'''
    sim_scores = list(enumerate(model[index]))
    sim_scores_sorted = sorted(sim_scores, key = lambda x:x[1], reverse=True)

    '''Finding and returning the location most similar to user's preferences'''
    best_fit_score = sim_scores_sorted[1]
    best_fit_index = best_fit_score[0]
    recommended_place = trial_model_df['place'].iloc[best_fit_index]

    return f'You should go to: {recommended_place}!'

In [232]:
find_rec()

'You should go to: San Diego, CA!'

In [268]:
indices['Austin, TX']

1

In [269]:
trial_model_df.iloc[1]['avg_monthly_price']

9816.145584725538

In [209]:
cs_df[33]

array([0.85100936, 0.86068596, 0.89646015, 0.90605552, 0.88156583,
       0.77336091, 0.84213213, 0.89275666, 0.89697403, 0.73681875,
       0.80636906, 0.88149077, 0.75397283, 0.80792062, 0.94371327,
       0.73909004, 0.82931122, 0.83609589, 0.94488851, 0.86343604,
       0.9479034 , 0.93593383, 0.88500789, 0.86587869, 0.71886936,
       0.94976773, 0.86161206, 0.90994135, 0.94569342, 0.926109  ,
       0.88057069, 0.79821859, 0.85312821, 1.        ])

In [198]:
trial_model_df.iloc[:, 1:]

,winter_avg_temp,summer_avg_temp,chain_ratio,walkability_idx,political_left_lean,political_right_lean,miscellaneous_cost_idx,avg_monthly_price
0,40.166667,70.200000,0.351600,9.382035,0.5970,0.3860,111.200000,5081.183654
1,53.400000,82.333333,0.381400,11.516379,0.7140,0.2640,105.900000,9816.145585
2,33.100000,72.233333,0.279600,14.936791,0.8060,0.1750,157.900000,6139.778716
3,31.100000,71.633333,0.327700,12.458517,0.7150,0.2630,176.700000,5535.792952
4,28.133333,73.933333,0.371700,13.635529,0.7420,0.2400,139.800000,5532.853672
5,32.866667,74.100000,0.501200,11.556746,0.6470,0.3340,107.500000,3835.510118
6,49.566667,82.666667,0.478300,11.899341,0.6490,0.3330,107.800000,5658.053279
7,33.166667,73.466667,0.327400,14.462231,0.7960,0.1820,152.200000,4313.462492
8,70.300000,82.533333,0.383700,12.445745,0.6450,0.3470,107.600000,5663.896761
9,49.166667,82.133333,0.559500,9.585953,0.4930,0.4910,109.200000,4079.534543


In [184]:
trial_model_df

,place,winter_avg_temp,summer_avg_temp,walkability_idx,chain_ratio,political_left_lean,political_right_lean,miscellaneous_cost_idx,avg_monthly_price
0,"Asheville, NC",40.166667,70.200000,9.382035,0.351600,0.5970,0.3860,111.200000,5081.183654
1,"Austin, TX",53.400000,82.333333,11.516379,0.381400,0.7140,0.2640,105.900000,9816.145585
2,"Boston, MA",33.100000,72.233333,14.936791,0.279600,0.8060,0.1750,157.900000,6139.778716
3,"Cambridge, MA",31.100000,71.633333,12.458517,0.327700,0.7150,0.2630,176.700000,5535.792952
4,"Chicago, IL",28.133333,73.933333,13.635529,0.371700,0.7420,0.2400,139.800000,5532.853672
5,"Columbus, OH",32.866667,74.100000,11.556746,0.501200,0.6470,0.3340,107.500000,3835.510118
6,"Dallas, TX",49.566667,82.666667,11.899341,0.478300,0.6490,0.3330,107.800000,5658.053279
7,"Denver, CO",33.166667,73.466667,14.462231,0.327400,0.7960,0.1820,152.200000,4313.462492
8,"Fort Lauderdale, FL",70.300000,82.533333,12.445745,0.383700,0.6450,0.3470,107.600000,5663.896761
9,"Fort Worth, TX",49.166667,82.133333,9.585953,0.559500,0.4930,0.4910,109.200000,4079.534543


In [101]:
user_input

[65,
 75,
 'I like to try new restaurants',
 'Neutral',
 'Moderate left lean preferred',
 5,
 6000]

In [117]:
test_dict = {'place': 'User Input', 'low_temp': 65, 'high_temp': 75, 'food_pref': 'I like to try new restaurants', 'walk_pref': 'Neutral', 'pol_pref': 'Moderate left lean preferred', 'misc_costs': 5, 'monthly_budget': 6000}

In [165]:
list(test_dict.values())

['User Input',
 65,
 75,
 'I like to try new restaurants',
 'Neutral',
 'Moderate left lean preferred',
 5,
 6000]

In [119]:
print(test_dict['food_pref'])

I like to try new restaurants


In [97]:
user_input

['User Input',
 65,
 75,
 'I like to try new restaurants',
 'Neutral',
 'Moderate left lean preferred',
 5,
 6000]

In [70]:
user_input.insert(0, 'User Input')

In [71]:
user_input

['User Input', 65, 75, 12.07544654388207, 0.25, 0.8, 0.2, 155, 6000]

In [11]:
walk_mean = trial_model_df['walkability_idx'].mean()
walk_std = trial_model_df['walkability_idx'].std()

In [12]:
walk_input_options = ['Not important at all', 'Not very important', 'Neutral', 'A little important', 'Very important']

In [13]:
walk_input_index = [walk_mean - (2*walk_std), walk_mean - (1*walk_std), walk_mean, walk_mean + (1*walk_std), walk_mean + (2*walk_std)]

In [14]:
walk_input_index

[7.077144672571502,
 9.576295608226786,
 12.07544654388207,
 14.574597479537353,
 17.073748415192636]

In [15]:
walkability_mapper = list(zip(walk_input_options, walk_input_index))
walkability_mapper

[('Not important at all', 7.077144672571502),
 ('Not very important', 9.576295608226786),
 ('Neutral', 12.07544654388207),
 ('A little important', 14.574597479537353),
 ('Very important', 17.073748415192636)]

In [16]:
trial_model_df.describe()

,winter_avg_temp,summer_avg_temp,walkability_idx,chain_ratio,political_left_lean,political_right_lean,miscellaneous_cost_idx,avg_monthly_price
count,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000
mean,46.471082,74.791385,12.075447,0.328676,0.701200,0.277730,129.344848,6387.315367
std,14.479006,6.471081,2.499151,0.100857,0.100124,0.100504,25.397655,2347.730358
min,17.985714,62.200000,4.986450,0.113600,0.489000,0.054000,92.900000,3437.974960
25%,35.533333,70.900000,11.519265,0.284800,0.645000,0.202000,107.600000,4606.418306
50%,47.033333,75.133333,12.445745,0.313500,0.714000,0.266500,124.000000,5663.896761
75%,51.333333,80.166667,13.635529,0.383700,0.779000,0.346000,152.900000,7618.616379
max,78.333333,87.200000,15.957258,0.559500,0.921000,0.491000,176.700000,14795.287657


In [17]:
user_input = ['User Input', 65, 75, 'I like to try new restaurants', 'Neutral', 'Moderate left lean preferred', 5, 6000]

# can we turn this into a dictionary?

In [18]:
user_input[5]

'Moderate left lean preferred'

In [19]:
for i in walkability_mapper:
    print(i[0])

Not important at all
Not very important
Neutral
A little important
Very important


In [20]:
pol_pref_options = ['Strong left lean preferred', 'Moderate left lean preferred', 'Moderate right lean preferred', 'Strong right lean preferred']
pol_pref_nums = [.9, .8, .6, .5]

pol_dem_mapper = list(zip(pol_pref_options, pol_pref_nums))
pol_dem_mapper

[('Strong left lean preferred', 0.9),
 ('Moderate left lean preferred', 0.8),
 ('Moderate right lean preferred', 0.6),
 ('Strong right lean preferred', 0.5)]

In [21]:
trial_model_df['chain_ratio'].std()

0.10085718873208491

In [22]:
[i[1] for i in pol_dem_mapper if user_input[5] == i[0]] 

[0.8]

In [23]:
for i in walkability_mapper:
    if user_input[4] == i[0]:
        print(i[1])
        user_walk_index = i[1]

12.07544654388207


In [24]:
if user_input[5] == 'No':
    user_pol_dem = .7
else:
    user_pol_dem = [i[1] for i in pol_dem_mapper if user_input[5] == i[0]]   

print(user_pol_dem[0])

0.8


In [25]:
user_input = ['User Input', 65, 75, 12.07544654388207, 0.25, 0.8, 0.2, 155, 6000]

In [26]:
len(user_input)

9

In [27]:
user_input_2 = ['User Input_2', 70, 80, 12.07544654388207, 0.30, 0.7, 0.3, 140, 6000]

In [28]:
user_input_3 = ['User Input_3', 65, 75, 14.574597479537353, 0.2, 0.85, 0.15, 135, 5500]

In [29]:
trial_model_df.loc[len(trial_model_df)] = user_input_3

/var/folders/td/_057xlcn5px5szx_85_2v2dc0000gn/T/ipykernel_1861/1424498399.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trial_model_df.loc[len(trial_model_df)] = user_input_3


In [30]:
trial_model_df.describe()

,winter_avg_temp,summer_avg_temp,walkability_idx,chain_ratio,political_left_lean,political_right_lean,miscellaneous_cost_idx,avg_monthly_price
count,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000
mean,47.016050,74.797521,12.148951,0.324891,0.705576,0.273974,129.511176,6361.217857
std,14.607755,6.372381,2.498037,0.101739,0.101844,0.101364,25.028679,2316.887820
min,17.985714,62.200000,4.986450,0.113600,0.489000,0.054000,92.900000,3437.974960
25%,35.883333,71.083333,11.528635,0.280900,0.645000,0.189250,107.650000,4636.817211
50%,47.733333,75.066667,12.452131,0.313150,0.714500,0.265250,128.600000,5660.975020
75%,52.508333,79.616667,13.757222,0.383125,0.782750,0.343000,152.725000,7490.267552
max,78.333333,87.200000,15.957258,0.559500,0.921000,0.491000,176.700000,14795.287657


In [31]:
trial_model_df

,place,winter_avg_temp,summer_avg_temp,walkability_idx,chain_ratio,political_left_lean,political_right_lean,miscellaneous_cost_idx,avg_monthly_price
0,"Asheville, NC",40.166667,70.200000,9.382035,0.3516,0.5970,0.3860,111.20,5081.183654
1,"Austin, TX",53.400000,82.333333,11.516379,0.3814,0.7140,0.2640,105.90,9816.145585
2,"Boston, MA",33.100000,72.233333,14.936791,0.2796,0.8060,0.1750,157.90,6139.778716
3,"Cambridge, MA",31.100000,71.633333,12.458517,0.3277,0.7150,0.2630,176.70,5535.792952
4,"Chicago, IL",28.133333,73.933333,13.635529,0.3717,0.7420,0.2400,139.80,5532.853672
5,"Columbus, OH",32.866667,74.100000,11.556746,0.5012,0.6470,0.3340,107.50,3835.510118
6,"Dallas, TX",49.566667,82.666667,11.899341,0.4783,0.6490,0.3330,107.80,5658.053279
7,"Denver, CO",33.166667,73.466667,14.462231,0.3274,0.7960,0.1820,152.20,4313.462492
8,"Fort Lauderdale, FL",70.300000,82.533333,12.445745,0.3837,0.6450,0.3470,107.60,5663.896761
9,"Fort Worth, TX",49.166667,82.133333,9.585953,0.5595,0.4930,0.4910,109.20,4079.534543


In [32]:
test = model_df[['place', 'price', 'pct_no_vehicle', 'cost_of_living', 'walkability_idx', 'chain_ratio']]

In [33]:
trial_model_df.columns

Index(['place', 'winter_avg_temp', 'summer_avg_temp', 'walkability_idx',
       'chain_ratio', 'political_left_lean', 'political_right_lean',
       'miscellaneous_cost_idx', 'avg_monthly_price'],
      dtype='object')

In [34]:
feature_cols = ['winter_avg_temp', 'summer_avg_temp', 'walkability_idx',
       'chain_ratio', 'political_left_lean', 'political_right_lean',
       'miscellaneous_cost_idx', 'avg_monthly_price']

sc = MinMaxScaler()
scaled_trial = sc.fit_transform(trial_model_df[feature_cols])

print(scaled_trial[:2])

[[0.36755307 0.32       0.40066191 0.53375196 0.25       0.7597254
  0.21837709 0.14468288]
 [0.58683816 0.80533333 0.59520954 0.60058309 0.52083333 0.4805492
  0.15513126 0.56159153]]


In [35]:
indices = pd.Series(trial_model_df.index, index=trial_model_df['place'])

cosine = cosine_similarity(scaled_trial)

def place_recommender(place_name, model=cosine):
    index = indices[place_name]

    scores = list(enumerate(model[index]))

    sim_scores = sorted(scores, key= lambda x:x[1], reverse=True)
    sim_scores = sim_scores[1:11]

    top_recs_index = [i[0] for i in sim_scores]

    top_places = trial_model_df['place'].iloc[top_recs_index]

    return top_places

In [36]:
print('Recommended Places:')
print(place_recommender('User Input_3', cosine).values)

Recommended Places:
['Oakland, CA' 'New Orleans, LA' 'Portland, OR' 'New York City, NY'
 'Washington DC, DC' 'Boston, MA' 'Newark, NJ' 'San Francisco, CA'
 'Los Angeles, CA' 'Denver, CO']


In [39]:
similarity_scores = sorted(list(enumerate(cosine[indices['San Francisco, CA']])), key= lambda x:x[1], reverse=True)

In [40]:
[i[0] for i in similarity_scores]

[26,
 29,
 22,
 21,
 30,
 2,
 33,
 27,
 32,
 7,
 28,
 3,
 18,
 4,
 14,
 19,
 11,
 25,
 17,
 31,
 23,
 20,
 1,
 8,
 0,
 6,
 16,
 5,
 10,
 24,
 13,
 15,
 12,
 9]

In [42]:
test['place'].iloc[[i[0] for i in similarity_scores]]

IndexError: positional indexers are out-of-bounds

In [43]:
list(enumerate(cosine[indices['San Francisco, CA']]))

[(0, 0.6193912742600769),
 (1, 0.6663204108168225),
 (2, 0.9357412021223088),
 (3, 0.8657516959803817),
 (4, 0.8395501792898792),
 (5, 0.5982463256055518),
 (6, 0.6021465000932469),
 (7, 0.9031081772719182),
 (8, 0.6470090012206144),
 (9, 0.39795593080947744),
 (10, 0.508528500018575),
 (11, 0.8026399584354277),
 (12, 0.4351730226961039),
 (13, 0.4970843593989582),
 (14, 0.8213301133178844),
 (15, 0.48973877309779135),
 (16, 0.5999687496506284),
 (17, 0.7483638945425544),
 (18, 0.8631268510337504),
 (19, 0.8089178836193812),
 (20, 0.7030865468392643),
 (21, 0.9416791504997746),
 (22, 0.9422408807616666),
 (23, 0.7171311974160652),
 (24, 0.5064231347577338),
 (25, 0.755175841221089),
 (26, 1.0000000000000002),
 (27, 0.9174870444316416),
 (28, 0.8899594919509353),
 (29, 0.9448331426029035),
 (30, 0.938926867318185),
 (31, 0.7249665443563678),
 (32, 0.905169599864383),
 (33, 0.9184636375596953)]

In [44]:
indices

place
Asheville, NC           0
Austin, TX              1
Boston, MA              2
Cambridge, MA           3
Chicago, IL             4
Columbus, OH            5
Dallas, TX              6
Denver, CO              7
Fort Lauderdale, FL     8
Fort Worth, TX          9
Hawaii, HI             10
Jersey City, NJ        11
Kauai, HI              12
Las Vegas, NV          13
Los Angeles, CA        14
Maui, HI               15
Nashville, TN          16
New Orleans, LA        17
New York City, NY      18
Newark, NJ             19
Oahu, HI               20
Oakland, CA            21
Portland, OR           22
Rhode Island, RI       23
Salem, OR              24
San Diego, CA          25
San Francisco, CA      26
San Mateo, CA          27
Santa Clara, CA        28
Santa Cruz, CA         29
Seattle, WA            30
Twin Cities, MN        31
Washington DC, DC      32
User Input_3           33
dtype: int64

In [45]:
cosine_distances(model_df.iloc[:, 2:])

array([[0.00000000e+00, 4.29554703e-06, 3.87045136e-06, ...,
        4.06008214e-06, 4.11825903e-06, 3.83185116e-06],
       [4.29554703e-06, 0.00000000e+00, 1.85363105e-08, ...,
        1.21248582e-08, 8.32707103e-09, 1.91021039e-08],
       [3.87045136e-06, 1.85363105e-08, 0.00000000e+00, ...,
        3.12543202e-09, 6.59206745e-09, 3.81653376e-10],
       ...,
       [4.06008214e-06, 1.21248582e-08, 3.12543202e-09, ...,
        0.00000000e+00, 5.97219252e-09, 4.69485928e-09],
       [4.11825903e-06, 8.32707103e-09, 6.59206745e-09, ...,
        5.97219252e-09, 0.00000000e+00, 7.30497374e-09],
       [3.83185116e-06, 1.91021039e-08, 3.81653376e-10, ...,
        4.69485928e-09, 7.30497374e-09, 0.00000000e+00]])

In [46]:
pairwise_distances(model_df.iloc[:, 2:], metric='cosine')

array([[4.44089210e-16, 4.29554703e-06, 3.87045136e-06, ...,
        4.06008214e-06, 4.11825903e-06, 3.83185116e-06],
       [4.29554703e-06, 4.44089210e-16, 1.85363105e-08, ...,
        1.21248582e-08, 8.32707103e-09, 1.91021039e-08],
       [3.87045136e-06, 1.85363105e-08, 0.00000000e+00, ...,
        3.12543202e-09, 6.59206745e-09, 3.81653376e-10],
       ...,
       [4.06008214e-06, 1.21248582e-08, 3.12543202e-09, ...,
        5.55111512e-16, 5.97219252e-09, 4.69485928e-09],
       [4.11825903e-06, 8.32707103e-09, 6.59206745e-09, ...,
        5.97219252e-09, 0.00000000e+00, 7.30497374e-09],
       [3.83185116e-06, 1.91021038e-08, 3.81653376e-10, ...,
        4.69485939e-09, 7.30497351e-09, 0.00000000e+00]])

In [47]:
similarity_matrix= cosine_similarity(model_df.iloc[:, 2:])

sys.getsizeof(similarity_matrix)

8840

In [48]:
similarity_matrix

array([[1.        , 0.9999957 , 0.99999613, ..., 0.99999594, 0.99999588,
        0.99999617],
       [0.9999957 , 1.        , 0.99999998, ..., 0.99999999, 0.99999999,
        0.99999998],
       [0.99999613, 0.99999998, 1.        , ..., 1.        , 0.99999999,
        1.        ],
       ...,
       [0.99999594, 0.99999999, 1.        , ..., 1.        , 0.99999999,
        1.        ],
       [0.99999588, 0.99999999, 0.99999999, ..., 0.99999999, 1.        ,
        0.99999999],
       [0.99999617, 0.99999998, 1.        , ..., 1.        , 0.99999999,
        1.        ]])

In [49]:
pd.DataFrame(cosine_similarity(model_df.iloc[:, 2:]), index = model_df['place'], columns=model_df['place'])

place,"Asheville, NC","Austin, TX","Boston, MA","Cambridge, MA","Chicago, IL","Columbus, OH","Dallas, TX","Denver, CO","Fort Lauderdale, FL","Fort Worth, TX",...,"Rhode Island, RI","Salem, OR","San Diego, CA","San Francisco, CA","San Mateo, CA","Santa Clara, CA","Santa Cruz, CA","Seattle, WA","Twin Cities, MN","Washington DC, DC"
place,,,,,,,,,,,,,,,,,,,,,
"Asheville, NC",1.000000,0.999996,0.999996,1.000000,0.999995,0.999996,0.999995,0.999996,0.999999,0.999996,...,0.999996,0.999999,0.999995,0.999996,0.999996,0.999995,0.999998,0.999996,0.999996,0.999996
"Austin, TX",0.999996,1.000000,1.000000,0.999996,1.000000,1.000000,1.000000,1.000000,0.999998,1.000000,...,1.000000,0.999999,1.000000,1.000000,1.000000,1.000000,0.999999,1.000000,1.000000,1.000000
"Boston, MA",0.999996,1.000000,1.000000,0.999997,1.000000,1.000000,1.000000,1.000000,0.999999,1.000000,...,1.000000,0.999999,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
"Cambridge, MA",1.000000,0.999996,0.999997,1.000000,0.999996,0.999996,0.999996,0.999997,0.999999,0.999996,...,0.999996,0.999999,0.999996,0.999997,0.999997,0.999996,0.999999,0.999997,0.999997,0.999997
"Chicago, IL",0.999995,1.000000,1.000000,0.999996,1.000000,1.000000,1.000000,1.000000,0.999998,1.000000,...,1.000000,0.999999,1.000000,1.000000,1.000000,1.000000,0.999999,1.000000,1.000000,1.000000
"Columbus, OH",0.999996,1.000000,1.000000,0.999996,1.000000,1.000000,1.000000,1.000000,0.999998,1.000000,...,1.000000,0.999999,1.000000,1.000000,1.000000,1.000000,0.999999,1.000000,1.000000,1.000000
"Dallas, TX",0.999995,1.000000,1.000000,0.999996,1.000000,1.000000,1.000000,1.000000,0.999998,1.000000,...,1.000000,0.999999,1.000000,1.000000,1.000000,1.000000,0.999999,1.000000,1.000000,1.000000
"Denver, CO",0.999996,1.000000,1.000000,0.999997,1.000000,1.000000,1.000000,1.000000,0.999999,1.000000,...,1.000000,0.999999,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
"Fort Lauderdale, FL",0.999999,0.999998,0.999999,0.999999,0.999998,0.999998,0.999998,0.999999,1.000000,0.999998,...,0.999998,1.000000,0.999998,0.999999,0.999999,0.999998,1.000000,0.999999,0.999999,0.999999


In [50]:
model_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 51 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   place                     33 non-null     object 
 1   price                     33 non-null     float64
 2   auto_accss_idx            33 non-null     float64
 3   pct_no_vehicle            33 non-null     float64
 4   pct_one_or_more_vehicles  33 non-null     float64
 5   transit_accss_idx         33 non-null     float64
 6   walkability_idx           33 non-null     float64
 7   cost_of_living            33 non-null     float64
 8   grocery_cost_idx          33 non-null     float64
 9   transportation_cost_idx   33 non-null     float64
 10  miscellaneous_cost_idx    33 non-null     float64
 11  population                33 non-null     int64  
 12  median_age                33 non-null     float64
 13  summer_comfort_index      33 non-null     float64
 14  winter_comfo

In [51]:
# We're seeing the most nulls in the neighborhood group column

model_df.isnull().sum().sum()

0

In [52]:
# Fortunately, we don't have much use for the columns that have null values, so we'll be dropping these and a few others
# Neighborhood group included here because we'll only need the neighborhood column for our analysis

airbnb.drop(columns=['name', 'host_id', 'host_name', 'neighbourhood_group', 'minimum_nights'], inplace=True) 
old_shape = airbnb.shape
old_shape

NameError: name 'airbnb' is not defined

In [ ]:
# Checking for and removing any duplicate listings in the data
# Keeping none as I don't feel that we could accurately/fairly determine which city to leave the listing in for

airbnb.drop_duplicates(subset=['id'], keep=False, inplace=True)
print(f'Total listings dropped: {old_shape[0] - airbnb.shape[0]}')

Total listings dropped: 10151


In [ ]:
airbnb['room_type'].value_counts()

Entire home/apt    202199
Private room        62695
Shared room          2765
Hotel room           1184
Name: room_type, dtype: int64